<a href="https://colab.research.google.com/github/bengy124/Bulk_RNA-Seq_Pipeline/blob/main/Bulk_RNA_Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bulk RNA-Seq workflow via Snakemake pipeline

In [15]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

Mounted at /content/gdrive/


#Installing Mamba

In [1]:
! wget -O miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
! chmod +x miniconda.sh
! bash ./miniconda.sh -b -f -p /usr/local
! rm miniconda.sh
! conda config --add channels conda-forge
! conda install -y mamba
! mamba update -qy --all
! mamba clean -qafy
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2023-12-18 11:38:10--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89026327 (85M) [application/x-sh]
Saving to: ‘miniconda.sh’

miniconda.sh        100%[===================>]  84.90M   126MB/s    in 0.7s    

2023-12-18 11:38:11 (126 MB/s) - ‘miniconda.sh’ saved [89026327/89026327]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2021.7.5=h06a4308_1
    - certifi==2021.5.30=py37h06a4308_0
    - cffi==1.14.6=py37h400218f_0
    - chardet==4.0.0=py37h06a4308_1003
    - conda-pa

# Installing packages

In [2]:
!mamba install -c bioconda snakemake sra-tools fastqc fastp STAR


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.27.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['snakemake', 'sra-tools', 'fastqc', 'fastp', 'star']

[+] 0.0s
bioconda/linux-64  ⣾  [+] 0.1s
bioc

# Obtaining fastq files

In [30]:
!touch obtain_fastq.sh
!chmod +x obtain_fastq.sh
!sed -i.bak 's/\r$//' obtain_fastq.sh

In [5]:
!rm -r rawReads/

In [6]:
!mkdir rawReads
!./obtain_fastq.sh SRP375527 rawReads/


2023-12-18T14:33:21 prefetch.3.0.9 int: string unexpected while executing query within virtual file system module - multiple response SRR URLs for the same service 's3'
2023-12-18T14:33:21 prefetch.3.0.9: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2023-12-18T14:33:22 prefetch.3.0.9: 1) Downloading 'SRR19237625'...
2023-12-18T14:33:22 prefetch.3.0.9: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2023-12-18T14:33:22 prefetch.3.0.9:  Downloading via HTTPS...
2023-12-18T14:34:43 prefetch.3.0.9:  HTTPS download succeed
2023-12-18T14:34:48 prefetch.3.0.9:  'SRR19237625' is valid
2023-12-18T14:34:48 prefetch.3.0.9: 1) 'SRR19237625' was downloaded successfully
2023-12-18T14:34:48 prefetch.3.0.9: 'SRP375527' has 0 unresolved dependencies
2023-12-18T14:34:48 prefetch.3.0.9: 1) Downloading 'SRR19237626'...
2023-12-18T14:34:48 prefetch.3.0.9: SRA Normalized

# Obtaining reference genome files

In [24]:
!mkdir ref_genome
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.29_GRCh38.p14/GCA_000001405.29_GRCh38.p14_genomic.fna.gz
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.29_GRCh38.p14/GCA_000001405.29_GRCh38.p14_genomic.gtf.gz
!gunzip GCA_000001405.29_GRCh38.p14_genomic.fna.gz GCA_000001405.29_GRCh38.p14_genomic.gtf.gz
!mv GCA_000001405.29_GRCh38.p14_genomic.fna ref_genome/
!mv GCA_000001405.29_GRCh38.p14_genomic.gtf ref_genome/

--2023-12-18 12:14:07--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.29_GRCh38.p14/GCA_000001405.29_GRCh38.p14_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.7, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 973047490 (928M) [application/x-gzip]
Saving to: ‘GCA_000001405.29_GRCh38.p14_genomic.fna.gz’

GCA_000001405.29_GR 100%[===================>] 927.97M  83.2MB/s    in 10s     

2023-12-18 12:14:17 (92.9 MB/s) - ‘GCA_000001405.29_GRCh38.p14_genomic.fna.gz’ saved [973047490/973047490]

--2023-12-18 12:14:17--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.29_GRCh38.p14/GCA_000001405.29_GRCh38.p14_genomic.gtf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.7, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ft

# Snakemake pipeline

In [23]:
!touch snakefile.py
!mkdir rawQC trimmedReads trimmedQC
!mkdir -p STAR/aligned

In [7]:
# Dry run
!snakemake -s snakefile.py -n

Building DAG of jobs...
Job stats:
job              count    min threads    max threads
-------------  -------  -------------  -------------
STAR                 2              1              1
STAR_index           1              1              1
all                  1              1              1
fastp                2              1              1
rawFastqc            4              1              1
trimmedFastqc        4              1              1
total               14              1              1


[Mon Dec 18 14:57:15 2023]
rule rawFastqc:
    input: rawReads/SRR19237626_1.fastq
    output: rawQC/SRR19237626_1_fastqc.zip, rawQC/SRR19237626_1_fastqc.html
    jobid: 4
    reason: Missing output files: rawQC/SRR19237626_1_fastqc.html, rawQC/SRR19237626_1_fastqc.zip
    wildcards: sra=SRR19237626, frr=1
    priority: 5
    resources: tmpdir=/tmp


[Mon Dec 18 14:57:15 2023]
rule fastp:
    input: rawReads/SRR19237625_1.fastq, rawReads/SRR19237625_2.fastq
    output: trimmedReads

In [8]:
!snakemake -s snakefile.py -j 10

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 10
Rules claiming more threads will be scaled down.
Job stats:
job              count    min threads    max threads
-------------  -------  -------------  -------------
STAR                 2             10             10
STAR_index           1             10             10
all                  1              1              1
fastp                2             10             10
rawFastqc            4              1              1
trimmedFastqc        4              1              1
total               14              1             10

Select jobs to execute...

[Mon Dec 18 14:57:17 2023]
rule rawFastqc:
    input: rawReads/SRR19237626_1.fastq
    output: rawQC/SRR19237626_1_fastqc.zip, rawQC/SRR19237626_1_fastqc.html
    jobid: 4
    reason: Missing output files: rawQC/SRR19237626_1_fastqc.zip, rawQC/SRR19237626_1_fastqc.html
    wildcards: sra=SRR19237626, frr=1
    priority: 5
    resources: tmpdir=/tmp


[Mon Dec 18 